# Setup

In [1]:
# Include these at the top of your notebook to automatically
# read saved changes to the imported modules below

%load_ext autoreload
%autoreload 2

In [2]:
# Import modules and local files here

from arctic_objects import *
from arctic_functions import *
import scipy.io as sio
import netCDF4 as nc
import numpy as np

In [3]:
# Paths to Data and Figures
MIT0_fp = '/ocean/jolerner/data/MITGCM/'

runs = ['CTL', 'OBS', 'REG', 'HI', 'LO']

NEMO_fp = '/ocean/jolerner/data/NEMO/'

ASTE_fp = '/ocean/jolerner/data/ASTE/'

WODC_fp = '/ocean/jolerner/data/WOD/'

SAVE_fp = '/ocean/jolerner/plots/'

# Define Callback Functions
contour={'func':potentialDensity, 'args':{'ref':0}}

In [4]:
# Load Grids and Regions
MIT0_grid = sio.loadmat(MIT0_fp + 'grid.mat', simplify_cells=True)['grid']
NEMO_grid = sio.loadmat(NEMO_fp + 'grid.mat', simplify_cells=True)['grid']
ASTE_grid = sio.loadmat(ASTE_fp + 'grid.mat', simplify_cells=True)['grid']

MIT0_reg = sio.loadmat(MIT0_fp + 'regions.mat', simplify_cells=True)['regions']
NEMO_reg = sio.loadmat(NEMO_fp + 'regions.mat', simplify_cells=True)['regions']
ASTE_reg = sio.loadmat(ASTE_fp + 'regions.mat', simplify_cells=True)['regions']
MIT0_reg = makeRegions(ASTE_reg, ASTE_grid, MIT0_grid)

In [5]:
# Load Transects

CBAR_coords = ([-144, 70], [25, 70])
FRAM_coords = ([-16.67, 80.67], [11.7, 79])
BFGY_coords = ([-144, 70], [-144, 83])

MIT0_CBAR = Transect('CANBAR Transect', MIT0_grid, CBAR_coords[0], CBAR_coords[1])
ASTE_CBAR = Transect('CANBAR Transect', ASTE_grid, CBAR_coords[0], CBAR_coords[1])
NEMO_CBAR = Transect('CANBAR Transect', NEMO_grid, CBAR_coords[0], CBAR_coords[1])

MIT0_FRAM = Transect('FRAM Transect', MIT0_grid, FRAM_coords[0], FRAM_coords[1])
ASTE_FRAM = Transect('FRAM Transect', ASTE_grid, FRAM_coords[0], FRAM_coords[1])
NEMO_FRAM = Transect('FRAM Transect', NEMO_grid, FRAM_coords[0], FRAM_coords[1])

MIT0_BFGY = Transect('Beaufort Gyre Transect', MIT0_grid, BFGY_coords[0], BFGY_coords[1])
ASTE_BFGY = Transect('Beaufort Gyre Transect', ASTE_grid, BFGY_coords[0], BFGY_coords[1])
NEMO_BFGY = Transect('Beaufort Gyre Transect', NEMO_grid, BFGY_coords[0], BFGY_coords[1])


# Data

To load in the data use: `data = sio.loadmat(PATH-TO-FILE, simplify_cells=True)[FIELD_NAME]`

Some files have multiple fields in them. For example, `CTL_DIFFzTH_2008-2012avg.mat` contains the fields for 
* Explicit Vertical Diffusive Heat Flux (`FIELD_NAME = 'Explicit'`)
* Implicit Vertical Diffusive Heat Flux (`FIELD_NAME = 'Implicit'`)
* Total Vertical Diffusive Heat Flux (`FIELD_NAME = 'Total'`)

To load the Total Vertical Diffusive Heat Flux from the MITGCM Control Run:

In [6]:
data = sio.loadmat(MIT0_fp + 'CTL/CTL_DIFFzTH_2008-2012avg.mat', simplify_cells=True)['Total']

This field uses the MITGCM grid (210 x 192 x 50) and regional logicals. To make a field object for the data:

In [7]:
field1 = Field('MITGCM Control', data['long_name'], data['units'], data['data'], MIT0_grid, MIT0_reg)

There are several ways to visualize the data:
* Vertical Regional Profiles (`FIELD.visualize_regional_profile()`)
* Regional Distributions (`FIELD.visualize_distributions()`)
* Contour Maps (`FIELD.visualize_maps`)
* Cross-sections of Transects (`TRANSECT.visualize_transect(FIELD)`)

Let's look at a vertical regional profile of the Vertical Diffusive Heat Flux from the MITGCM Control Run:

In [8]:
fig = field1.visualize_regional_profile(scale='symlog', show=False)

Computing weighted means for MITGCM Control Total Vertical Diffusive Heat Flux


Next, let's look at the Vertical Diffusive Heat Flux from the MITGCM Control Run along the CANBAR transect:

In [9]:
fig = MIT0_CBAR.visualize_transect(field1, scale='symlog', show=False)

If instead, we want to look at the same field from the ASTE Control Run

In [10]:
data = sio.loadmat(ASTE_fp + 'CTL_DIFFzTH_2008-2012avg.mat', simplify_cells=True)['Total']
field2 = Field('ASTE Control', data['long_name'], data['units'], data['data'], ASTE_grid, ASTE_reg)
fig = ASTE_CBAR.visualize_transect(field2, scale='symlog', show=False)

Now, let's look at an anomaly plot of two fields, 
comparing the Potential Temperature of the MITGCM Control Run and the ASTE Control Run:

In [11]:
data1 = sio.loadmat(MIT0_fp + 'CTL/CTL_theta_2008-2012avg.mat', simplify_cells=True)['theta']
data2 = sio.loadmat(ASTE_fp + 'CTL_theta_2008-2012avg.mat', simplify_cells=True)['theta']
field3 = Field('ASTE - MITGCM Control', data1['long_name'], data1['units'], data1['data']-data2['data'], MIT0_grid, MIT0_reg)

ValueError: operands could not be broadcast together with shapes (210,192,50) (450,270,50) 

The data is not the same shape, so we will need to resample the data to fit on the same grid using

`resample(old_data, old_grid, new_grid)`

In [12]:
field3 = Field('MITGCM - ASTE Control', data1['name'], data1['units'], 
               data1['data']-resample(data2['data'], ASTE_grid, MIT0_grid), MIT0_grid, MIT0_reg)
fig = MIT0_CBAR.visualize_transect(field3, show=False)

/home/jolerner/arcticMix/arctic_functions.py:299: RuntimeWarning: Mean of empty slice
  new_data = np.nanmean(flat_data[idx], axis=1)


There is another type of plot called a volume census that utilizes salinity and temperature

In [13]:
SALT = Field('MITGCM Control', 'Practical Salinity', 'PSU', 
             sio.loadmat(MIT0_fp + 'CTL/CTL_salinity_2008-2012avg.mat', simplify_cells=True)['salinity']['data'], MIT0_grid, MIT0_reg)
HEAT = Field('MITGCM Control', 'Potential Temperature', '˚C', 
             sio.loadmat(MIT0_fp + 'CTL/CTL_theta_2008-2012avg.mat', simplify_cells=True)['theta']['data'], MIT0_grid, MIT0_reg)
fig = volumeCensus(SALT, HEAT, contour=contour, range=((26,36),(-2.5,7.5)), show=False)